Data Engineering - Group 1
Healthcare - Flu Tracker
---

 

## Extraction

In [19]:
import pandas as pd

# Step 1: Load CSV data into Pandas DataFrames for the two CSV files
file_path_ilinet = "ILINet.csv"
file_path_clinical_labs = "WHO_NREVSS_Clinical_Labs.csv"

df_ilinet = pd.read_csv(file_path_ilinet)
df_clinical_labs = pd.read_csv(file_path_clinical_labs)

# Display the first few rows of each DataFrame
print("First few rows of ILINet DataFrame:")
print(df_ilinet.head())

print("\nFirst few rows of WHO_NREVSS_Clinical_Labs DataFrame:")
print(df_clinical_labs.head())

First few rows of ILINet DataFrame:
  REGION TYPE      REGION  YEAR  WEEK % WEIGHTED ILI %UNWEIGHTED ILI AGE 0-4  \
0      States     Alabama  2010    40              X         2.13477       X   
1      States      Alaska  2010    40              X        0.875146       X   
2      States     Arizona  2010    40              X        0.674721       X   
3      States    Arkansas  2010    40              X        0.696056       X   
4      States  California  2010    40              X         1.95412       X   

  AGE 25-49 AGE 25-64 AGE 5-24 AGE 50-64 AGE 65 ILITOTAL NUM. OF PROVIDERS  \
0         X         X        X         X      X      249                35   
1         X         X        X         X      X       15                 7   
2         X         X        X         X      X      172                49   
3         X         X        X         X      X       18                15   
4         X         X        X         X      X      632               112   

  TOTAL PATIEN

In [20]:
print("Column names of df_ilinet:", df_ilinet.columns)


Column names of df_ilinet: Index(['REGION TYPE', 'REGION', 'YEAR', 'WEEK', '% WEIGHTED ILI',
       '%UNWEIGHTED ILI', 'AGE 0-4', 'AGE 25-49', 'AGE 25-64', 'AGE 5-24',
       'AGE 50-64', 'AGE 65', 'ILITOTAL', 'NUM. OF PROVIDERS',
       'TOTAL PATIENTS'],
      dtype='object')


In [21]:
df_ilinet_selected = df_ilinet[['REGION', 'AGE 0-4', 'AGE 25-49', 'AGE 25-64', 'AGE 5-24', 'AGE 50-64', 'AGE 65', 'ILITOTAL', 'NUM. OF PROVIDERS', 'TOTAL PATIENTS']]

# WHO_NREVVS_Clinical_Labs DataFrame
df_clinical_labs_selected = df_clinical_labs[['REGION TYPE', 'TOTAL SPECIMENS', 'TOTAL A', 'TOTAL B']]

# Display the selected DataFrames
print("Selected Columns from ILINet DataFrame:")
print(df_ilinet_selected.head())

print("\nSelected Columns from WHO_NREVVS_Clinical_Labs DataFrame:")
print(df_clinical_labs_selected.head())

Selected Columns from ILINet DataFrame:
       REGION AGE 0-4 AGE 25-49 AGE 25-64 AGE 5-24 AGE 50-64 AGE 65 ILITOTAL  \
0     Alabama       X         X         X        X         X      X      249   
1      Alaska       X         X         X        X         X      X       15   
2     Arizona       X         X         X        X         X      X      172   
3    Arkansas       X         X         X        X         X      X       18   
4  California       X         X         X        X         X      X      632   

  NUM. OF PROVIDERS TOTAL PATIENTS  
0                35          11664  
1                 7           1714  
2                49          25492  
3                15           2586  
4               112          32342  

Selected Columns from WHO_NREVVS_Clinical_Labs DataFrame:
  REGION TYPE  TOTAL SPECIMENS  TOTAL A  TOTAL B
0    National            12029       84       43
1    National            13111      116       54
2    National            13441       97       52
3 

## Transformation